In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
dataset = pd.read_csv("./student_cgpa_iq_passfail_300.csv")
dataset.head(3)

,CGPA,IQ,Result
0,7.37,131,1
1,5.75,82,0
2,7.90,118,1


In [3]:
def perceptron(X, y, lr=0.1, epochs=10000):
    features = np.asarray(X, dtype=float)
    sample = np.asarray(y, dtype=int)
    
    weight = np.zeros(features.shape[1], dtype=float)
    bias = 0.0
    
    for epoch in range(epochs):
        errors = 0
        for i in range(len(features)):
            x_i = features[i]  # [CGPA, IQ]
            y_i = sample[i]    # label
            
            net_input = np.dot(x_i, weight) + bias
            
            # step function
            y_pred = 1 if net_input >= 0 else 0
            error = y_i - y_pred
            
            if error != 0:
                errors += 1
                weight = weight + (lr * error * x_i)
                bias = bias + (lr * error)
        
        # Early stopping if converged
        if errors == 0:
            print(f"Converged at epoch {epoch}")
            break
    
    return weight, bias

In [4]:
X_raw = dataset[["CGPA", "IQ"]].values.astype(float)
y = dataset["Result"].values.astype(int)
X_min = X_raw.min(axis=0)
X_max = X_raw.max(axis=0)
X = (X_raw - X_min) / (X_max - X_min)

In [5]:
X_min = X_raw.min(axis=0)
X_max = X_raw.max(axis=0)
print(X_min, X_max)

[ 3.92 70.  ] [ 10. 144.]


In [7]:
X = (X_raw - X_min) / (X_max - X_min)
print(X)

[[0.56743421 0.82432432]
 [0.30098684 0.16216216]
 [0.65460526 0.64864865]
 [0.69243421 0.40540541]
 [0.12171053 0.45945946]
 [0.25       0.25675676]
 [0.53125    0.40540541]
 [0.44407895 0.74324324]
 [0.50328947 0.59459459]
 [0.33881579 0.82432432]
 [0.68092105 0.71621622]
 [0.65953947 0.56756757]
 [0.51973684 0.82432432]
 [0.72861842 0.56756757]
 [0.59868421 0.63513514]
 [0.33717105 0.41891892]
 [0.57894737 0.48648649]
 [0.31743421 0.33783784]
 [0.67927632 0.67567568]
 [0.49671053 0.22972973]
 [0.47039474 0.63513514]
 [0.37171053 0.43243243]
 [0.74835526 0.71621622]
 [0.47532895 0.62162162]
 [0.42269737 0.83783784]
 [0.4375     0.62162162]
 [0.61184211 0.14864865]
 [0.57894737 0.45945946]
 [0.58881579 0.22972973]
 [0.59210526 0.36486486]
 [0.92927632 0.78378378]
 [0.42598684 0.60810811]
 [0.40625    0.33783784]
 [0.34539474 0.27027027]
 [0.62828947 0.47297297]
 [0.72861842 0.22972973]
 [0.48355263 0.33783784]
 [0.34046053 0.54054054]
 [0.34375    0.7027027 ]
 [0.63486842 0.59459459]


In [5]:
weight, bias = perceptron(X, y)

In [6]:
print(f"\nTrained model:")
print(f"  Weights: {weight}")
print(f"  Bias: {bias}")


Trained model:
  Weights: [0.48404605 0.17162162]
  Bias: -0.30000000000000004


In [7]:
def predict_one(cgpa, iq):
    """
    Predict if a student will pass or fail
    """
    # Normalize the input using the same min/max from training
    x_raw = np.array([float(cgpa), float(iq)])
    x_normalized = (x_raw - X_min) / (X_max - X_min)
    
    # Make prediction
    net = np.dot(x_normalized, weight) + bias
    y_pred = 1 if net >= 0 else 0
    
    return "PASS" if y_pred == 1 else "FAIL"

In [8]:
print(f"CGPA=7.5, IQ=110: {predict_one(7.5, 110)}")

CGPA=7.5, IQ=110: PASS


In [9]:
print(f"CGPA=7.5, IQ=110: {predict_one(6.5, 150)}")

CGPA=7.5, IQ=110: PASS


In [10]:
predict_one(8.2, 163)

'PASS'

In [30]:
import json
from datetime import datetime

In [34]:
model_data = {
    "model_info": {
        "model_type": "Perceptron",
        "algorithm": "Binary Classification",
        "training_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    },
    "parameters": {
        "weights": weight.tolist(),  # Convert numpy array to list
        "bias": float(bias)
    },
    "normalization": {
        "X_min": X_min.tolist(),
        "X_max": X_max.tolist(),
        "features": ["CGPA", "IQ"]
    },
    "training_config": {
        "learning_rate": 0.1,
        "max_epochs": 10000,
        "num_samples": len(dataset)
    },
    "labels": {
        "0": "FAIL",
        "1": "PASS"
    }
}

In [35]:
with open('perceptron_model_compact.json', 'w') as f:
    json.dump(model_data, f)